# refInSingleLine()

In [2]:
import re
import collections
    
def patterns(text):
    patterns=['see section','in section', 
              'of section', 'under section','within section',
               'with section','by section', 'that section',
              'omit section','also section','a section', 'to section',
              'and section', 'which section','amend section'
              'on section','than section','which section','the section','after section',
              'those sections','the section','before section','no section',
              'those section','or section','instead section','than section','despite section',
              'where section','for section', ' section ']

    for y in patterns:
            if(y in text and not ( ("of the section" in text) or  ("c." in text)) ):
                dependency=findDependency(text.lower(), y)
                if(dependency is not None and len(dependency)>0):
                    return findDependency(text.lower(), y);

    if('Section' in text and (' to ' not in text)):
        return findMatch(text, 'Section')
    elif(text.startswith('section ')):
        return findMatch(text, 'section')
    elif(text.startswith('sections ')):
        text=text.replace("sections ","")
        if("to " in text):
            return findToDependency(text);
        else:
            return findMatch(text, 'sections')
    elif('Sections' in text and (' to ' not in text)):
        return findMatch(text, 'Sections')
    else:
        if((str(" to ") in text) and 'Parts' not in text):
            return findToDependency(text[0:text.index(' to ')+10]) #local text

                    

def addPattern(array, text):
    if (isinstance(text, list)==True):
        for x in text:
            array.append(str(x))
        return array
    elif(text is not None):
        return array.append(str(text))
    else:
        return array


In [8]:
def findMatch(text, pattern):
    text=subsectionBracketsRef(text)
    if(pattern in text):
        text[text.index(pattern)+len(pattern):]
        match=re.sub("[^0-9]", "", text[text.index(pattern)+len(pattern):])    
        if(match in text):
            numberOne=text[text.index(match):text.index(match)+len(match)]
            numberTwo=text[text.index(match):text.index(match)+len(match)+1]
            if(numberOne in numberTwo and len(numberOne)>0):
                match=numberTwo    
            return clearSection(match)
        else:
                # TODO consider later pattern occurrences - give details how many such cases we have. 
            match=re.sub("[^0-9][1]", "", text[text.index(pattern)+len(pattern)+1:])
            if(" " in match and " or " not in match and " to " not in match):
                return clearSection(match[0:match.index(" ")])
            else: 
                if(("ofthe" not in match) and "of the" not in text and ('Act' not in text)):
                    return clearSection(match)
    else:
        return text

def findDependency(text, pattern):    
    match=findMatch(text,pattern)
    if(clearSection(str(match)) in text and match!=None):
        if((str(" and ") in text) and notSubesctions(text) and ofThe(text)):
            return findAndDependency(text, pattern)         
        elif((str(" to "+pattern) in text)):
            return findToDependency(match)
        elif((str(" or "+pattern) in text)):
            return findOrDependency_1(text, pattern)
        elif(" of section" in text):
            text=text[text.index('of section')+11:]
            text=text[0:text.index(' ')]
            return clearSection(text)    
        else:
            if(("of the" not in text)  and ('Act' not in text)):
                return clearSection(str(match))
    else:
        if((str(" to ") in text)):
            return findToDependency(text[0:text.index(' to ')+10]) #local text
        elif((str(" followed ") in text)):
            return findDependency(text[0:text.index(' followed ')], pattern) 
        elif((str(" and ") in text)): 
            return findDependency(text[0:text.index(' and ')], pattern) 
        if((str(" or ") in text) and (str(" or after") not in text)):
            return findOrDependency_1(text, pattern)
        else:
            if("(" in text and pattern in text):
                    match=re.sub("[^0-9]\(\d+\)", "", text[text.index(pattern)+len(pattern):])
                    if(")" in match):
                        return str(match[1:match.index(")")+1:])                        
            else:
                if(("ofthe" not in match) and "of the" not in text and ('Act' not in text)):
                    if(text.startswith('Section') or text.startswith('section')):
                        return str(match).lower().replace("section ","")
                    else:
                        return str(match)
            
def subsectionBracketsRef(text):
    if("(" in text):
        index_1=text.index("(")
        index_2=text.index(")")
        if(index_1<index_2):
            if( (index_1+2==index_2 or index_1+3==index_2) and notSubesctions(text)):
                return text[0:index_1]
            else:
                return text
        else:
             return text
    else:
        return text
    
def notSubesctions(text):
    if('subsection' not in text and 'Subsection' not in text):
        return True
    else:
        return False

def ofThe(text):
    if('of the' not in text):
        return True
    else:
        return False

def findToDependency(text):
    match=re.findall(r'\d+ to \d+', text)
    if(len(match)>0):
        text=re.findall(r'\d+ to \d+', text)[0]
        to=re.findall(r'\d+', text)
        return list(range(int(to[0]),int(to[1])+1))

def clearParentheses(text):
    return text[0:text.index("(")];

def subString(text, character):
    if(character in text):
        return text[0:text.index(character)];
    else:
        return text;
    
def removePrefix(text, character):
    return text[text.index(character):];

def removeSpace(text):
    return text.replace(" ","")

def clearSection(text):
    if(',' in text):
        text=subString(text, ',')
    if('“' in text):
        text=subString(text, '“')
    if('(' in text):
        text=subString(text, '(')
    if(')' in text):
        text=subString(text, ')')
    if(' ' in text):
        text=removeSpace(text)
    if('.' in text):
        text=subString(text, '.')
    if(any(char.isdigit() for char in text)):
        return text
    else:
        return ''

def keepNumbers(text):
    return  re.sub("[^0-9][1]", "", text)

def clearORText(text):
    match=re.sub("[a-zA-Z]", "", text)
    match=re.sub("\((\d+)\).", "", match)
    match=re.sub(" ", "", match)
    match=''.join(char for char in match if char.isalnum())
    return match;

def findOrDependency(text, pattern):
    temp=text[text.index(pattern)+len(pattern)+1:text.index(' or ')]
    if("(" in temp):
        return [clearParentheses(temp), clearORText(text[text.index(' or ')+4:])]
    else: 
        return [temp, clearORText(text[text.index(' or ')+4:])]

def findOrDependency_1(text, pattern):
    match=re.sub("[^0-9] or [^0-9]", "", text[text.find(pattern)+len(pattern):])
    if(' or ' in match and ofThe(match)):
        numberTwo=match[match.index(' or ')+4:]
        if(' ' in numberTwo):
            numberTwo=numberTwo[0:numberTwo.index(' ')]
        else:
            numberTwo=clearSection(numberTwo)
        return [ match[0:match.index(' or ')], numberTwo]
    else:
        if(any(char.isdigit() for char in match) and ofThe(match) ):
            return clearSection(match)
        else:
            text=text[0:text.index(' or ')-2]
            text=[text.index(' section ')+9]
            return text

def findAndDependency(text, pattern):
    match=re.sub("[^0-9] and [^0-9]", "", text[text.index(pattern)+len(pattern)+1:])
    if(' and ' in match):
        numberTwo=match[match.index(' and ')+5:]
        if('of that Act' not in match and 'of that Act' not in numberTwo):
            return [ clearSection(match[0:match.index(' and ')]), clearSection(numberTwo)]
    else:
        if('of that Act' not in text):
            return clearSection(text[text.index(pattern)+len(pattern)+1:])

def forSectionOfTheAct(text):
      match=re.sub("For Section [^0-9] of the", "", text)
      return clearSection(match)



# Testing scenarios, each variable section_{Number}_{subsectionNumber} corresponds to the content of section with number Number and subsection number: subsectionNumber. That is, section_74_16 variable corresponds to the content  of section 74, subsection 16.

In [7]:
def unit_tests():
    section_74_16="The appropriate national authority may by regulations make such provision as it considers appropriate for supplementing the provisions of this section and section 73, and in particular"
    assert patterns(section_74_16) == '73'
    section_11_5="unless previously terminated by subsection (7) or revoked under section 93 or 93A, continues in force for the period that is so specified or determined."
    assert patterns(section_11_5) == [' 93', '93a,']
    section_259_1="This section sets out when persons are to be treated for the purposes of section 254 as occupying a building or part of a building as their only or main residence."
    assert patterns(section_259_1)
    section_85_3="Sections 87 to 90 deal with applications for licences, the granting or refusal of licences and the imposition of licence conditions"
    assert patterns(section_85_3)==[87, 88, 89, 90]
    section_66_3C="A person is not a fit and proper person for the purposes of section 64(3)(b) or (d) if a banning order under section 16 of the Housing and Planning Act 2016 is in force against the person"
    assert patterns(section_66_3C)=='64'
    section_130_a="a lease or licence in which the local housing authority was substituted for another party by virtue of section 124(4) has effect with the substitution of the original party, or his successor in title, for the authority; and"
    assert patterns(section_130_a)=='124'
    section_113_1="A local housing authority who have made an interim management order in respect of a house under any provision of section 102 other than subsection (7A) of that section] (“the IMO”)"
    assert patterns(section_113_1)=='102'
    section_230_8="Section 229(5) applies also for the purposes of this section and Schedule 13."
    assert patterns(section_230_8)=='229'
    section_43_5_b="sections 32 to 36 (enforcement);"
    assert patterns(section_43_5_b) == [32, 33, 34, 35, 36]

    return 'unit_tests pass'

print(unit_tests())

unit_tests pass
